In [ ]:
import yaml
import pickle
import os.path as osp
import subprocess
from urllib.parse import urlparse
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def retrieve_url(url, dest_path, force_download=False):
    if not osp.exists(dest_path) or force_download:
        target_extension = osp.splitext(dest_path)[1]
        url_extension = osp.splitext(urlparse(url).path)[1]
        if target_extension != url_extension:
            print("Warning: file extension from url does not match destination file extension")
        subprocess.call(f"wget -O {dest_path}  {url}", shell=True)
        assert osp.exists(dest_path)
        print(f"Successfully downloaded {url} to {dest_path}")
    else:
        print(f"Target data already exists at {dest_path}")

In [ ]:
retrieve_url("https://demo.openwfm.org/web/data/fmda/dicts/fmda_nw_202401-05_f05.pkl", "data/fmda_nw_202401-05_f05.pkl")

In [ ]:
def read_pkl(file_path):
    with open(file_path, 'rb') as file:
        print(f"loading file {file_path}")
        d = pickle.load(file)
    return d

In [ ]:
dat = read_pkl("data/test_CA_202401.pkl")

In [ ]:
dat['NV040_202401']['RAWS']['fm']

In [ ]:
dat['NV040_202401']['RAWS'].keys()

In [ ]:
dat['NV040_202401']['RAWS']['time_raws'][1]

In [ ]:
from utils import filter_nan_values, time_intp, str2time

In [ ]:
time_raws = str2time(dat['NV040_202401']['RAWS']['time_raws'])
time_hrrr = str2time(dat['NV040_202401']['HRRR']['time'])
fm = dat['NV040_202401']['RAWS']['fm']

In [ ]:
len(time_raws) == len(fm)

In [ ]:
plt.plot(time_raws, fm)

In [ ]:
time_hrrr[0]

In [ ]:
time_hrrr[-1]

In [ ]:
time_raws[0]

In [ ]:
time_raws[1]

In [ ]:
xx = time_intp(time_raws,fm,time_hrrr)

In [ ]:
len(xx)

In [ ]:
yy, zz = filter_nan_values(time_raws,fm)

In [ ]:
len(yy)

In [ ]:
len(zz)

In [ ]:
t1_stamps = np.array([t.timestamp() for t in yy])
t2_stamps = np.array([t.timestamp() for t in time_hrrr])

In [ ]:
v2_interpolated = np.interp(t2_stamps, t1_stamps, zz)

In [ ]:
len(t1_stamps)

In [ ]:
plt.plot(v2_interpolated)

In [ ]:
def read_yml(yaml_path, subkey=None):
    with open(yaml_path, 'r') as file:
        d = yaml.safe_load(file)
        if subkey is not None:
            d = d[subkey]
    return d

In [ ]:
data_params = read_yml("params_data.yaml")
data_params

In [ ]:
def filter_fmc(x, data_params=data_params):
    # Vector to track which values of input x are set to na
    na_vec = np.zeros_like(x)
    
    # Filter extreme lows
    condition = x<= data_params['min_fm']
    x[condition] = np.nan
    na_vec[np.where(condition)] = 1
    
    # Filter extreme highs
    condition = x>= data_params['max_fm']
    x[condition] = np.nan
    na_vec[condition] = 1
    
    return x, na_vec

In [ ]:
def filter_rain(r, data_params=data_params):
    # Vector to track which values of input x are set to na
    na_vec = np.zeros_like(r)
    
    # Filter extreme lows
    condition = r<= data_params['min_rain']
    r[condition] = np.nan
    na_vec[np.where(condition)] = 1
    
    # Filter extreme highs
    condition = r>= data_params['max_rain']
    r[condition] = np.nan
    na_vec[condition] = 1    

    return r, na_vec

In [ ]:
yy = np.array([.01, 0, 10, 14, .01, 700])

In [ ]:
filter_fmc(yy)

In [ ]:
filter_rain(np.array([-1, 1111, 10, 5, 2, 0.01]))

In [ ]:
def foo():
    d = read_pkl()
    dat = format_fmda_data()
    dat = filter_data(dat)
    return dat

In [ ]:
def format_fmda_data(d):
    # Inputs:
    #    d: (dict) fmda dictionary, output of process in wrfxpy

    return d